In [1]:
import sys
import os
import torch
import torch.nn as nn
import cv2
import numpy as np
import pandas as pd
import time
import tkinter as tk
from PIL import Image,ImageTk
from torchvision.ops import nms
sys.path.append('C:/Users/Mehmet/yolov7')

from models.experimental import attempt_load
from utils.torch_utils import select_device

weights_path = r'C:\Users\Mehmet\yolov7\yolov7.pt'
device = select_device('') 
model = attempt_load(weights_path, map_location=device)
classes = [
    "person",
    "bicycle",
    "car",
    "motorcycle",
    "airplane",
    "bus",
    "train",
    "truck",
    "boat",
    "traffic light",
    "fire hydrant",
    "stop sign",
    "parking meter",
    "bench",
    "bird",
    "cat",
    "dog",
    "horse",
    "sheep",
    "cow",
    "elephant",
    "bear",
    "zebra",
    "giraffe",
    "backpack",
    "umbrella",
    "handbag",
    "tie",
    "suitcase",
    "frisbee",
    "skis",
    "snowboard",
    "sports ball",
    "kite",
    "baseball bat",
    "baseball glove",
    "skateboard",
    "surfboard",
    "tennis racket",
    "bottle",
    "wine glass",
    "cup",
    "fork",
    "knife",
    "spoon",
    "bowl",
    "banana",
    "apple",
    "sandwich",
    "orange",
    "broccoli",
    "carrot",
    "hot dog",
    "pizza",
    "donut",
    "cake",
    "chair",
    "couch",
    "potted plant",
    "bed",
    "dining table",
    "toilet",
    "TV",
    "laptop",
    "mouse",
    "remote",
    "keyboard",
    "cell phone",
    "microwave",
    "oven",
    "toaster",
    "sink",
    "refrigerator",
    "book",
    "clock",
    "vase",
    "scissors",
    "teddy bear",
    "hair drier",
    "toothbrush"
]

C:\Users/Mehmet/yolov7\models\experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(w, map_location=map_location)  # load


Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


C:\Users\Mehmet\anaconda3\envs\pytorch\lib\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
def predicted_objects(frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame,(640,640))
        frame = frame/255
        input= torch.from_numpy(frame).float()
        input=input.permute(2, 0, 1).unsqueeze(0)
        input=input.to(device)
        with torch.no_grad():
            preds = model(input)[0]  
            for boxs in preds: #tahminler 25200 uzunluktaki boxs dizisine bölünür
                item=[]
                for box in boxs:
                    box=box.cpu().numpy()
                    if box[4]>0.8:#nesnenin 4 indeksteki değeri güven aralığını ifade eder %70 doğruluk payı threshold ekliyoruz
                        x,y,w,h=box[0:4]
                        c=box[5:].argmax()
                        s=box[4]
                        item.append([[x,y,x + w, y + h],c,s])
                item=np.array(item,dtype=object)
                if len(item[:, 0]) > 0:
                    boxes = torch.tensor(item[:,0].tolist(),  dtype=torch.float32)
                    scores = torch.tensor(item[:,1].tolist(), dtype=torch.float32)
                    classes = torch.tensor(item[:, 1].tolist(),dtype=torch.int64)
                    indices = nms(boxes, scores, iou_threshold=0.9)#%90 benzerliğe sahip nesnelerden güven skoru düşük olanları nms fonksiyonuyla eledik bunun içinde numpy verilerimizi torch türüne çevirdik
                    filtered_boxes = item[:,0][indices]         #elde edilen indisler bütün indislerden elenmiş olanların çıkarılmış halidir
                    filtered_scores = item[:,2][indices]        #bu indislerin değerlerinide filtrelenmiş olarak tekrardan ele alıyoruz
                    filtered_classes = item[:, 1][indices]
                    all_predictions=np.column_stack((filtered_boxes, filtered_scores, filtered_classes))
                    return all_predictions #bütün tahminleri birleştirip döndürüyoruz

In [9]:
def detect(cam_path):
    video =cv2.VideoCapture(cam_path)
    while True:
        back,frame=video.read()
        if back==False:
            break
        else:
            predictions=predicted_objects(frame) #nesne sayısı uzunluğunda liste halinde 3 elemanlı listeden oluşan(1.konum 2.score 3.sınıf) bir dizi döndürür
            for pred in predictions:
                x1,y1,x2,y2=pred[0]
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 100, 0), 2)
                text=("Class : "+str(classes[pred[2]])+" Score : "+str(pred[1]))
                cv2.putText(frame, text, (int(x1)-10,int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 2)
                cv2.imshow('Car1',frame)
                if cv2.waitKey(1)==ord('q'):
                    break
    video.release()
    cv2.destroyAllWindows()

In [ ]:
cam_path=r'C:\Users\Mehmet\Desktop\traffic\cam1.mp4'
detect(cam_path)